In [1]:
import pathlib

import joblib
import numpy as np
import pandas as pd
import pycytominer

In [2]:
train_test_wells_path = pathlib.Path(
    "../data_splits/train_test_wells.parquet"
).resolve()

predictions_save_path = pathlib.Path(
    "../results/predicted_terminal_profiles_from_all_time_points.parquet"
).resolve()

profile_data_path = pathlib.Path(
    "../../data/CP_scDINO_features/combined_CP_scDINO_norm_fs_aggregated.parquet"
).resolve()
terminal_column_names = pathlib.Path("../results/terminal_columns.txt").resolve(
    strict=True
)
terminal_column_names = [
    line.strip() for line in terminal_column_names.read_text().splitlines()
]

data_split_df = pd.read_parquet(train_test_wells_path)
df = pd.read_parquet(profile_data_path)
metadata_cols = [cols for cols in df.columns if "Metadata" in cols]
features_cols = [cols for cols in df.columns if "Metadata" not in cols]
features_cols = features_cols
aggregate_df = pycytominer.aggregate(
    population_df=df,
    strata=["Metadata_Well", "Metadata_Time"],
    features=features_cols,
    operation="median",
)


metadata_df = df[metadata_cols]
metadata_df = metadata_df.drop_duplicates(subset=["Metadata_Well", "Metadata_Time"])
metadata_df = metadata_df.reset_index(drop=True)
aggregate_df = pd.merge(
    metadata_df, aggregate_df, on=["Metadata_Well", "Metadata_Time"]
)
print(aggregate_df.shape)
aggregate_df.head()

(390, 2343)


,Metadata_Well,Metadata_Time,Metadata_dose,Metadata_number_of_singlecells,Metadata_plate,Metadata_compound,Metadata_control,Cells_AreaShape_Compactness_CP,Cells_AreaShape_Eccentricity_CP,Cells_AreaShape_Extent_CP,...,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO
0,C-02,0.0,0.0,178.0,1,Staurosporine,negative,-0.225866,0.362134,0.195628,...,0.123994,-0.169231,-0.000171,-0.000714,0.219109,0.060974,0.073445,-0.099431,-0.202662,0.069599
1,C-02,1.0,0.0,181.0,1,Staurosporine,negative,-0.101367,0.212559,0.160123,...,-0.110603,0.076735,0.057752,0.065120,-0.253076,0.088509,-0.046158,0.118233,-0.090497,-0.116096
2,C-02,10.0,0.0,168.0,1,Staurosporine,negative,0.018758,0.438709,0.017725,...,-0.077459,-0.076050,0.179852,0.079523,-0.534580,0.115206,0.033339,0.179140,-0.031637,-0.142134
3,C-02,11.0,0.0,174.0,1,Staurosporine,negative,-0.041979,0.460182,-0.009282,...,-0.121237,-0.075781,0.197840,0.229394,-0.400539,0.074929,-0.064636,0.225648,-0.063553,-0.139911
4,C-02,12.0,0.0,175.0,1,Staurosporine,negative,0.045544,0.423336,0.057030,...,0.035974,-0.044032,0.490040,0.287544,-0.584386,0.145022,0.071476,0.225753,-0.012832,-0.152288


In [3]:
models_path = pathlib.Path("../models/").resolve(strict=True)
models = pathlib.Path(models_path).glob("*.joblib")
models_dict = {
    "model_name": [],
    "model_path": [],
    "shuffled": [],
    "feature": [],
}

for model_path in models:
    models_dict["model_name"].append(model_path.name)
    models_dict["model_path"].append(model_path)
    models_dict["shuffled"].append(
        "shuffled" if "shuffled" in model_path.name else "not_shuffled"
    )
    models_dict["feature"].append(
        model_path.name.split("singlefeature")[1].strip(".joblib").strip("_")
        if "singlefeature" in model_path.name
        else "all_terminal_features"
    )

In [4]:
# map the train/test wells to the aggregate data
aggregate_df["Metadata_data_split"] = aggregate_df["Metadata_Well"].map(
    data_split_df.set_index("Metadata_Well")["data_split"]
)
data_split = aggregate_df.pop("Metadata_data_split")
aggregate_df.insert(0, "Metadata_data_split", data_split)
aggregate_df["Metadata_Time"] = aggregate_df["Metadata_Time"].astype(float)
# drop NaN values in the terminal columns
aggregate_df = aggregate_df.dropna(subset="Metadata_data_split")
aggregate_df["Metadata_data_split"].unique()

array(['train', 'test'], dtype=object)

In [5]:
# if the data_split is train and the time is not 12 then set to non_trained_pair
# where 12 is the last time point
aggregate_df["Metadata_data_split"] = aggregate_df.apply(
    lambda x: (
        "non_trained_pair"
        if (x["Metadata_data_split"] == "train" and x["Metadata_Time"] != 12.0)
        else x["Metadata_data_split"]
    ),
    axis=1,
)

In [6]:
metadata_columns = [x for x in aggregate_df.columns if "metadata" in x.lower()]
aggregate_features_df = aggregate_df.drop(columns=metadata_columns, errors="ignore")

In [7]:
results_dict = {}
for i, model_name in enumerate(models_dict["feature"]):
    model = joblib.load(models_dict["model_path"][i])
    if models_dict["feature"][i] != "all_terminal_features":
        print(models_dict["feature"][i])
        predicted_df = pd.DataFrame(
            model.predict(aggregate_features_df),
            columns=[models_dict["feature"][i]],
        )
    else:
        print("all_terminal_features")
        predicted_df = pd.DataFrame(
            model.predict(aggregate_features_df),
            columns=terminal_column_names,
        )
    predicted_df[metadata_columns] = aggregate_df[metadata_columns]
    predicted_df["shuffled"] = models_dict["shuffled"][i]
    # drop nan value
    predicted_df = predicted_df.dropna()

    # check if a key for the feature already exists in results_dict
    if f"{models_dict['feature'][i]}" in results_dict:
        temporary_df = pd.concat(
            [results_dict[f"{models_dict['feature'][i]}"], predicted_df],
            ignore_index=True,
            sort=False,
        )
        results_dict[f"{models_dict['feature'][i]}"] = temporary_df
    else:
        results_dict[f"{models_dict['feature'][i]}"] = predicted_df

    print(results_dict[f"{models_dict['feature'][i]}"].shape)

Terminal_Cytoplasm_Intensity_MaxIntensity_AnnexinV
(390, 10)
Terminal_Cytoplasm_Intensity_MaxIntensity_AnnexinV
(780, 10)
all_terminal_features
(390, 519)
Terminal_Cytoplasm_Intensity_IntegratedIntensity_AnnexinV
(390, 10)
Terminal_Cytoplasm_Intensity_IntegratedIntensity_AnnexinV
(780, 10)
all_terminal_features
(780, 519)


In [8]:
for model in results_dict.keys():
    save_path = pathlib.Path(f"../results/{model}.parquet").resolve()
    save_path.parent.mkdir(parents=True, exist_ok=True)
    results_dict[model].to_parquet(save_path, index=False)